In [1]:
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
import plotly.express as px

from mainnet_launch.constants import *

from mainnet_launch.pages.rebalance_events.rebalance_events import _load_full_rebalance_event_df, make_expoded_box_plot

rebalance_df = _load_full_rebalance_event_df(AUTO_DOLA)
rebalance_df

2025-06-04 17:52:32.505 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-04 17:52:32.799 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:32.878 
  command:

    streamlit run /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-04 17:52:32.879 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:32.879 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:32.880 No runtime found, using MemoryCacheStorageManager
2025-06-04 17:52:33.385 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:33.386 Thread 'Thread-4

2025-06-04 17:52:33,614 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-04 17:52:33,615 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-04 17:52:33,759 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-04 17:52:33,759 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-04 17:52:33,903 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-04 17:52:33,904 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-04 17:52:34,050 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-04 17:52:34,051 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

2025-06-04 17:52:34.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:34.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:34.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:34.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:34.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-06-04 17:52:34,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-04 17:52:34,434 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-06-04 17:52:34,435 INFO sqlalchemy.engine.Engine [cached since 0.3842s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x11fb9fc40>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-06-04 17:52:34,436 INFO sqlalchemy.engine.Engine SELECT
    destination_tokens.*,
    tokens.symbol,
    tokens.decimals
FROM destination_tokens
JOIN tokens
  ON destination_t

2025-06-04 17:52:34.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-04 17:52:34.727 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-06-04 17:52:34,801 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-04 17:52:34,802 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-06-04 17:52:34,802 INFO sqlalchemy.engine.Engine [cached since 0.7519s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x11fad4160>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-06-04 17:52:34,803 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM destinations
            WHERE destinations.chain_id = 1
        
2025-06-04 17:52:34,8

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,...,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,swap_cost_in_bps_of_value_out,swap_cost_in_bps_of_NAV,from_idle,swap_cost_in_bps_of_value_out_from_idle,swap_cost_in_bps_of_NAV_from_idle,swap_cost_in_bps_of_value_out_not_from_idle,swap_cost_in_bps_of_NAV_not_from_idle
datetime,,,,,,,,,,,,,,,,,,,,,
2025-05-31 12:16:11+00:00,0x5ec201906fb3404bc1c6ca20b64878db450f288f1436...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748693691_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x9828b1a3742457D70cE428ddd3Aa51f84ffb37F2,260.711443,261.299545,260.711443,260.470158,...,5.986495,0.241285,9.254866,5.986495,1.496909,True,5.986495,1.496909,0.0,0.0
2025-06-01 01:56:11+00:00,0xfd99d0c3a72b198506caef16fd73a1d3e9461f69e3a8...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748742897_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x9828b1a3742457D70cE428ddd3Aa51f84ffb37F2,250.000000,250.588078,250.000000,249.874656,...,4.617265,0.125344,5.013769,4.617265,1.107178,True,4.617265,1.107178,0.0,0.0
2025-06-01 19:56:11+00:00,0x598beaa475b95a70e398e864e545a607cb6944ebab72...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748807696_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x9828b1a3742457D70cE428ddd3Aa51f84ffb37F2,250.000000,250.669514,250.000000,249.985003,...,3.278576,0.014997,0.599900,3.278576,0.786224,True,3.278576,0.786224,0.0,0.0
2025-06-02 22:04:47+00:00,0x3589558715a108df5e64ee343b3360bedf8c4d542c79...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748901807_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x9828b1a3742457D70cE428ddd3Aa51f84ffb37F2,20000.000000,20048.773378,20000.000000,19975.992145,...,5.743572,24.007855,12.003927,5.743572,4.357181,True,5.743572,4.357181,0.0,0.0
2025-06-03 03:03:35+00:00,0x87753fb83f4f460b4c24ce084fe202abeb68e4858700...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748919730_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x9828b1a3742457D70cE428ddd3Aa51f84ffb37F2,40000.000000,40045.397846,40000.000000,39933.329519,...,15.936186,66.670481,16.667620,15.936186,0.643326,True,15.936186,0.643326,0.0,0.0
2025-06-03 04:34:59+00:00,0x272e39d795a4a38e6cc22ec9e54deb92fa068c83304d...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748925231_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x116b3e86f2b04c21605c5fC4b95Ba6A82dBF8F3E,507586.651647,448213.886034,507586.651647,507510.513649,...,26.482906,76.137998,1.500000,26.482906,12.502765,True,26.482906,12.502765,0.0,0.0
2025-06-03 05:52:47+00:00,0x7ed68b1503242dd0419df2e2af3608e3ad36da7e9d39...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748929907_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x116b3e86f2b04c21605c5fC4b95Ba6A82dBF8F3E,253793.325823,224104.483853,253793.325823,253755.256825,...,10.479051,38.068999,1.500000,10.479051,2.473019,True,10.479051,2.473019,0.0,0.0
2025-06-03 08:04:35+00:00,0x92945b1955089f6bc009754f5f0e88a72ac5b9823dd7...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748937807_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x9828b1a3742457D70cE428ddd3Aa51f84ffb37F2,50000.000000,50053.933314,50000.000000,49960.269585,...,13.419549,39.730415,7.946083,13.419549,0.504321,True,13.419549,0.504321,0.0,0.0
2025-06-03 12:04:35+00:00,0x692e3b3232da729451ac45ae2628d1968e992c401a41...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748952206_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x9828b1a3742457D70cE428ddd3Aa51f84ffb37F2,50000.000000,50116.633595,50000.000000,50024.974176,...,0.927859,-24.974176,-4.994835,0.927859,0.032420,True,0.927859,0.032420,0.0,0.0


In [2]:
rebalance_df.to_csv(
    "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/mainnet_launch/working_data/rebalance_csvs/AUTO_DOLA_rebalance_df_swap_costs.csv"
)

In [3]:
import plotly.io as pio

pio.templates.default = None

In [16]:
px.bar(
    rebalance_df.resample("1d").sum()[["swap_cost_in_bps_of_NAV_from_idle", "swap_cost_in_bps_of_NAV_not_from_idle"]]
)

In [29]:
rebalance_df.groupby(["move_name"])["spot_slippage_bps"].mean().sort_values().reset_index()

,move_name,spot_slippage_bps
0,autoDOLA -> savedola,2.725424
1,autoDOLA -> reusdsdola,5.453234
2,autoDOLA -> sDOLA,10.470460


In [18]:
rebalance_df.groupby(["destination_in", "destination_out"])["spot_slippage_bps"].mean().sort_values().reset_index()

,destination_in,destination_out,spot_slippage_bps
0,0xF4CeDDf4e2AEc1aa6b4ef2c64739eCaC5C948fA2,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,2.725424
1,0x9828b1a3742457D70cE428ddd3Aa51f84ffb37F2,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,5.453234
2,0x116b3e86f2b04c21605c5fC4b95Ba6A82dBF8F3E,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,10.470460


In [32]:
rebalance_df[rebalance_df["move_name"] == "autoDOLA -> sDOLA"]

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,...,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,swap_cost_in_bps_of_value_out,swap_cost_in_bps_of_NAV,from_idle,swap_cost_in_bps_of_value_out_from_idle,swap_cost_in_bps_of_NAV_from_idle,swap_cost_in_bps_of_value_out_not_from_idle,swap_cost_in_bps_of_NAV_not_from_idle
datetime,,,,,,,,,,,,,,,,,,,,,
2025-06-03 04:34:59+00:00,0x272e39d795a4a38e6cc22ec9e54deb92fa068c83304d...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748925231_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x116b3e86f2b04c21605c5fC4b95Ba6A82dBF8F3E,507586.651647,448213.886034,507586.651647,507510.513649,...,26.482906,76.137998,1.5,26.482906,12.502765,True,26.482906,12.502765,0.0,0.0
2025-06-03 05:52:47+00:00,0x7ed68b1503242dd0419df2e2af3608e3ad36da7e9d39...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748929907_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x116b3e86f2b04c21605c5fC4b95Ba6A82dBF8F3E,253793.325823,224104.483853,253793.325823,253755.256825,...,10.479051,38.068999,1.5,10.479051,2.473019,True,10.479051,2.473019,0.0,0.0
2025-06-03 14:04:35+00:00,0x5b1cb3152d7d4e0b321521e0edd71b0f340e4b64b1f5...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1748959407_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x116b3e86f2b04c21605c5fC4b95Ba6A82dBF8F3E,254700.377680,224889.880882,254700.377680,254662.172623,...,7.765689,38.205057,1.5,7.765689,1.382145,True,7.765689,1.382145,0.0,0.0
2025-06-04 16:04:23+00:00,0x2d0b3ed18d1dd395743ae7c50e915a711f249c350794...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,1,rebalance_plan_1749053006_0x79eB84B5E30Ef2481c...,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d,0x116b3e86f2b04c21605c5fC4b95Ba6A82dBF8F3E,532669.114002,470222.748015,532669.114002,532589.213635,...,-2.845808,79.900367,1.5,-2.845808,-0.541617,True,-2.845808,-0.541617,0.0,0.0


In [33]:
int(1e18)

1000000000000000000

In [38]:
convert_to_shares = 882835246576454774
preview_deposit = 882835292565254565

10_000 * ((convert_to_shares - preview_deposit) / 1e18)

-0.00045988799791

In [39]:
((convert_to_shares - preview_deposit) / 1e18)

-4.5988799791e-08

In [19]:
px.bar(rebalance_df["spot_slippage_bps"])

In [20]:
px.bar(rebalance_df.resample("1d").sum()["spot_swap_cost"])

In [21]:
px.bar(
    rebalance_df.resample("1d").sum()[["swap_cost_in_bps_of_NAV_from_idle", "swap_cost_in_bps_of_NAV_not_from_idle"]]
)

In [22]:
px.bar(rebalance_df.resample("1d").sum()["spot_swap_cost"].rolling(28).sum())

In [23]:
10_000 * 2_000 / 20_000_000

1.0

In [24]:
px.bar(rebalance_df.resample("1d").sum()["spot_swap_cost"])

In [25]:
rebalance_df["move_name"].value_counts()

move_name
autoDOLA -> reusdsdola    16
autoDOLA -> savedola       6
autoDOLA -> sDOLA          4
Name: count, dtype: int64

In [26]:
# need date filtering
by_move_name = rebalance_df.groupby("move_name")[["safe_value_out", "safe_swap_cost"]].sum()
by_move_name["all_time_safe_slippage"] = 10_000 * by_move_name["safe_swap_cost"] / by_move_name["safe_value_out"]

px.bar(by_move_name["all_time_safe_slippage"])

px.scatter(by_move_name, x="safe_value_out", y="all_time_safe_slippage", color=by_move_name.index)

In [27]:
px.scatter(by_move_name, x="safe_value_out", y="all_time_safe_slippage", color=by_move_name.index)